In [ ]:
import pandas as pd
from numpy.random import choice
from my_pickle import unpickle_it
import matplotlib.pyplot as plt

In [ ]:
user_df = unpickle_it('user_df')
convo_df = unpickle_it('convo_df')

print("Make sure we're not useing ECT data")
print(pd.to_datetime(convo_df.timestamp.max()*1000000))

In [ ]:
print("Troubleshoot")
print(32436)
print(len(user_df))
print(len(convo_df))

# CELLS TO BE COPY-PASTED INTO .PY FILE

In [ ]:
# are user ids unique?
print(len(user_df.index))
print(len(user_df.index.unique())) 

In [ ]:
def remove_bad_uids(df, user_df):
    ''' 
    removes and rows of the df that have uids not in uids
    '''
    uids = set(user_df.index)
    df['flag1'] = df.first_uid.apply(lambda x: x not in uids)
    df['flag2'] = df.second_uid.apply(lambda x: x not in uids)
    df['flag'] = df.flag1 | df.flag2
    
    df = df.drop(df[df.flag].index)

    df = df.drop(['flag1','flag2','flag'], axis=1)
    return df
    

convo_df = remove_bad_uids(convo_df, user_df)  
len(convo_df)

In [ ]:
def get_rent_range(first_user, second_user):
    max1 = first_user.maxCost
    max2 = second_user.maxCost
    min1 = first_user.minCost
    min2 = second_user.minCost
    if max1*max2*min1*min2 > 0:
        upper = min(first_user.maxCost,second_user.maxCost)
        lower = max(first_user.minCost,second_user.minCost)
        if upper-lower > 0: return upper-lower
        else: return 0
    else: return 175

In [ ]:
yes1 = '0a9yOPKFSH'
yes2 = '013LzOrVju'
no1 = '0UBLgJIHgz'
no2 = '0EWNOmyQmW'
low = '01DE0NCjwh'
high = '02GDyQPLII'
print(get_rent_range(user_df.loc[yes1], user_df.loc[yes2])) # should be 150
print(get_rent_range(user_df.loc[yes1], user_df.loc[no1]))  # should be 175
print(get_rent_range(user_df.loc[no2], user_df.loc[no1]))   # should be 175
print(get_rent_range(user_df.loc[low], user_df.loc[high]))  # should be 0

In [ ]:
def feature_time(df, user_df):

    ad,sg,ro,sr,sc,sn,ss,sa,rs = [],[],[],[],[],[],[],[],[]

    for index, row in df.iterrows():
        first = user_df.loc[str(row.first_uid)]
        second = user_df.loc[str(row.second_uid)]
        ad.append(abs(first.age - second.age))
        sg.append(first.gender == second.gender)
        ro.append(get_rent_range(first, second))
        sr.append(first.inRelationship == second.inRelationship)
        sc.append(first.isClean == second.isClean)
        sn.append(first.isNight == second.isNight)
        ss.append(first.isStudent == second.isStudent)
        sa.append(first.attractiveness)
        rs.append(second.selectivity)

    df['age_dif'] = ad
    df['same_gender'] = sg
    df['rent_overlap'] = ro
    df['same_relate'] = sr
    df['same_clean'] = sc
    df['same_night'] = sn
    df['same_student'] = ss
    df['sender_attractiveness'] = sa
    df['receiver_selectivity'] = rs

    return df

In [ ]:
X = feature_time(convo_df, user_df)

In [ ]:
X.head()

In [ ]:
user_df['cost_range'] = user_df.maxCost-user_df.minCost
user_df.cost_range.hist(bins=range(0,2000,50))
plt.show()
user_df.head(2)

In [ ]:
def get_average_overlap(df,samples):
    # to find average overlap of two users' rent ranges
    # comes out to about $175
    s = 0
    for n in range(samples):
        two_users = choice(user_df[user_df.cost_range>0].index,2)
        overlap = user_df.loc[two_users,'maxCost'].min()-user_df.loc[two_users,'minCost'].max()
        if overlap < 0: overlap = 0
        s += overlap
    return s/samples

In [ ]:
X = feature_time(convo_df, user_df)

In [ ]:
print(len(X))